In [7]:
import feedparser
import requests
from bs4 import BeautifulSoup
import time

# --- Configuration ---
USERNAME = "pamudu1111" # Using a public handle for testing
RSS_URL = f"https://medium.com/feed/@{USERNAME}"
PROFILE_URL = f"https://medium.com/@{USERNAME}"

# --- Method 1: The RSS Approach (Clean & Structured) ---

print(f"--- Testing RSS Fetch ({RSS_URL}) ---")
start_time = time.time()

# 1. Parse the feed
feed = feedparser.parse(RSS_URL)

if not feed.entries:
    print("No entries found.")
    exit(1)

# 2. Extract data (Standardized keys: title, link, published)
print(f"Found {len(feed.entries)} entries.")
latest_post = feed.entries[1]
print(len)

print(f"✅ Success ({time.time() - start_time:.4f}s)")
print(f"Title:     {latest_post.title}")
print(f"Link:      {latest_post.link}")
print(f"Published: {latest_post.published}")
# RSS often gives the full content in 'summary' or 'content'
print(f"Content Preview: {latest_post.summary[:100]}...")
print("-" * 30)


--- Testing RSS Fetch (https://medium.com/feed/@pamudu1111) ---
Found 5 entries.
<built-in function len>
✅ Success (0.7324s)
Title:     Deep Learning Approach for Generalizable AI Image Detection
Link:      https://medium.com/@pamudu1111/technical-report-generalizable-ai-image-detection-8b0d901f3c2a?source=rss-8693d059d7ab------2
Published: Wed, 26 Nov 2025 16:50:54 GMT
Content Preview: <figure><img alt="" src="https://cdn-images-1.medium.com/max/1024/1*LfeG-re1yYPgzRIFj9uRYA.png" /></...
------------------------------


In [8]:
import feedparser
import json

# Global config for the tools
RSS_URL = "https://medium.com/feed/@pamudu1111"

def list_articles(limit=5):
    """
    Tool 1: Fetches metadata for the latest articles.
    Use this to see what topics are available.
    """
    feed = feedparser.parse(RSS_URL)
    
    # We only want the metadata, not the full content
    articles_metadata = []
    
    for entry in feed.entries[:limit]:
        articles_metadata.append({
            "title": entry.title,
            "date": entry.published,
            "link": entry.link,  # This acts as the ID
            "id": entry.id       # Alternative ID
        })
        
    # Return as a JSON string so the LLM can read it easily
    return json.dumps(articles_metadata, indent=2)

def get_article_content(target_link):
    """
    Tool 2: Fetches the FULL content of a specific article.
    Input: 'target_link' (found from list_articles tool).
    """
    feed = feedparser.parse(RSS_URL)
    
    # Find the specific article matching the link
    # (Functional filtering)
    matching_entry = next((entry for entry in feed.entries if entry.link == target_link), None)
    
    if not matching_entry:
        return "Error: Article not found. Please check the link."
    
    # Medium RSS usually puts full HTML content in 'content' list
    # If not, it falls back to 'summary'
    if 'content' in matching_entry:
        full_text = matching_entry.content[0].value
    else:
        full_text = matching_entry.summary
        
    return full_text


In [9]:

# --- Simulation of the Agent's Flow ---

# Step 1: User asks "What did I write about recently?"
# Agent calls list_articles()
print("🤖 Agent calls Tool 1 (List)...")
menu = list_articles(limit=3)
print(menu) 
# Agent reads the JSON... sees a title like "Understanding AI Agents"

# Step 2: User says (or Agent decides) "Tell me more about that AI Agents one."
# Agent grabs the 'link' from the JSON and calls Tool 2
print("\n🤖 Agent calls Tool 2 (Read)...")
# (Simulating the agent picking the first link from the menu)
first_link = json.loads(menu)[0]['link'] 
content = get_article_content(first_link)

print(f"Content length retrieved: {len(content)} characters")
print(f"Snippet: {content[:100]}...")

🤖 Agent calls Tool 1 (List)...
[
  {
    "title": "The Remote Work Paradox: A Quantitative Analysis of Professional Work-Life Balance in 2025",
    "date": "Sun, 21 Dec 2025 17:54:05 GMT",
    "link": "https://medium.com/@pamudu1111/the-remote-work-paradox-a-quantitative-analysis-of-professional-work-life-balance-in-2025-b35912fd240e?source=rss-8693d059d7ab------2",
    "id": "https://medium.com/p/b35912fd240e"
  },
  {
    "title": "Deep Learning Approach for Generalizable AI Image Detection",
    "date": "Wed, 26 Nov 2025 16:50:54 GMT",
    "link": "https://medium.com/@pamudu1111/technical-report-generalizable-ai-image-detection-8b0d901f3c2a?source=rss-8693d059d7ab------2",
    "id": "https://medium.com/p/8b0d901f3c2a"
  },
  {
    "title": "LLMs in Practice: How I Choose the Right Model",
    "date": "Sat, 26 Jul 2025 13:49:03 GMT",
    "link": "https://medium.com/@pamudu1111/llms-in-practice-how-i-choose-the-right-model-f87a49f2b861?source=rss-8693d059d7ab------2",
    "id": "https